In [1]:
%load_ext autoreload
%autoreload 2

import numpy as np
import os
import scipy.linalg

from graph_utils import graph_gen, find_alpha_lower_bound
from problem import Synthetic
from plot_utils import MyFigure

from algorithms import IPLUX, UDC, dual_subgradient
# Set up the logger to print info messages for understandability.
import logging
import sys
logging.basicConfig(level=logging.INFO, stream=sys.stdout, format='')


In [2]:
# generate problem

# network

num_node = 20
num_edge = 26

# np.random.seed(1)

graph_gen(num_node, num_edge, B=1)

net_dir = f'data/graph/N{num_node}E{num_edge}'
network = np.load(f'{net_dir}/subgraph_W.npy')
print(network.shape)
# print(network)

# problem data

parameters = {}
parameters['N'] = num_node
parameters['d'] = 3 # dimension of x_i's
parameters['m'] = 1 # number jof inequality constraints
parameters['p'] = 5 # number of equality constraints

prob = Synthetic(parameters)
# prob = Synthetic(parameters, debug=True)

prob.gen()
prob.load()
alpha_lower = find_alpha_lower_bound(parameters, prob)
print(f'x* {prob.x_star}')
print(f'||x*|| {np.linalg.norm(prob.x_star)}\n')

Graph with 20 nodes and 26 edges


C:\Users\xtq\AppData\Roaming\Python\Python311\site-packages\networkx\drawing\nx_pylab.py:315: UserWarning: 

The arrowsize keyword argument is not applicable when drawing edges
with LineCollection.

To make this warning go away, either specify `arrows=True` to
force FancyArrowPatches or use the default values.
Note that using FancyArrowPatches may be slow for large graphs.

  draw_networkx_edges(G, pos, arrows=arrows, **edge_kwds)


(20, 20)
generating a Synthetic problem: N=20, d=3, m=1, p=5
Q: (20, 3), P: (20, 3, 3)
A: (20, 5, 3)
a: (20, 3), c: (20,)
aa: (20, 3), cc: (20,)
x* (20, 3), f* -0.4565561073830914
generated problem saved in data/problem/Synthetic/N20

loading a Synthetic problem, N=20
problem loaded:
Q: (20, 3), P: (20, 3, 3)
A: (20, 5, 3)
a: (20, 3), c: (20,)
aa: (20, 3), ca: (20,)
x_star (20, 3)
opt_val -0.4565561073830914
x* [[ 5.50513429e-12 -1.79741804e-11 -1.25862957e-01]
 [ 4.63363469e-02 -6.78205363e-10 -1.69060094e-11]
 [-6.45839964e-12  1.35031488e-11 -6.98922920e-11]
 [-3.01035380e-11 -3.77818321e-11 -2.68543190e-10]
 [-4.58371218e-01  2.36048903e-01  2.27570498e-01]
 [ 7.22422110e-11 -6.18900918e-02  1.86445353e-11]
 [-1.12893041e-10 -5.37898488e-11 -1.17178202e-01]
 [-1.81856984e-10 -1.99789543e-01 -6.38696018e-11]
 [ 6.09053807e-11 -6.95485673e-12  8.93642894e-11]
 [ 3.35886373e-11  2.72708729e-12 -1.63429767e-02]
 [ 1.45810436e-01 -4.97196905e-10  6.35055444e-12]
 [ 4.18350630e-11  2.194

In [2]:
# load instance 
instance_name = 'instance1_N20E40'
# instance_name = 'instance2_N20E26'
instance_dir = 'instance/' + instance_name

# network
network = np.load(instance_dir + '/graph/subgraph_W.npy')
num_node = network.shape[0]
print(network.shape)

# problem data
parameters = {}
parameters['N'] = num_node
parameters['d'] = 1 # temporary, reset in load()
parameters['m'] = 1 # temporary
parameters['p'] = 1 # temporary

prob = Synthetic(parameters)
prob.save_dir = instance_dir + '/problem'
prob.load()

alpha_lower = find_alpha_lower_bound(parameters, prob)
print(f'x* {prob.x_star}')
print(f'||x*|| {np.linalg.norm(prob.x_star)}\n')


(20, 20)
loading a Synthetic problem, N=20
problem loaded:
Q: (20, 3), P: (20, 3, 3)
A: (20, 5, 3)
a: (20, 3), c: (20,)
aa: (20, 3), ca: (20,)
x_star (20, 3)
opt_val -1.7937811849951981
x* [[ 1.69192667e-08 -4.54451916e-02 -1.25326407e-02]
 [-1.97438972e-11  2.94708451e-10  1.96457124e-01]
 [-1.55498077e-01 -1.81713964e-10 -4.30739243e-02]
 [ 2.21417334e-10  6.41377197e-11 -2.57323583e-01]
 [ 5.46491556e-01 -3.09078549e-01 -2.25841238e-10]
 [-7.95790832e-10  4.77051274e-01 -9.77296145e-02]
 [ 2.30311170e-01  9.67073127e-10 -4.08278630e-02]
 [ 4.41710887e-10  5.03547130e-01  2.55567666e-02]
 [-4.78912681e-01 -1.55547314e-09  3.47456666e-10]
 [ 1.16595532e-10 -9.51339774e-11 -4.60428545e-01]
 [ 4.42020248e-10 -4.19241406e-01  1.76861413e-01]
 [-1.46017592e-09  6.03865377e-10 -1.04286398e-09]
 [-3.70053317e-01  6.45150319e-01  3.92380391e-10]
 [ 1.95357925e-10 -2.86013131e-10 -4.44910108e-10]
 [-1.46178792e-08 -3.47711897e-01  8.41473423e-10]
 [-8.31629811e-10 -1.66205909e-01  1.02254197e

In [24]:
# IPLUX

# Instance 2 best parameters:
# IPLUX: a20_r4 130.59s

MAX_ITER = 2000

log_dir = 'log'
if not os.path.exists(log_dir):
    os.makedirs(log_dir)

alpha_lower = find_alpha_lower_bound(parameters, prob)
# alpha_choice = [alpha_lower, alpha_lower*5, alpha_lower*25]
alpha_choice = [alpha_lower]

rho_choice = [1, 5, 25]
# rho_choice = [5]

color = ['b', 'g', 'r', 'c', 'm', 'y', 'k']
linestyle = ['-', '--', '-.', ':', '']

obj_err_figure = MyFigure(filename='obj_err', 
                            xlabel=r'$\mathrm{iteration}$ $k$', 
                            ylabel='objective error',
                            yscale='log')
cons_vio_figure = MyFigure(filename='cons_vio', 
                            xlabel=r'$\mathrm{iteration}$ $k$', 
                            ylabel='constraint violation',
                            yscale='log')
x_dis_figure = MyFigure(filename='x_distance', 
                            xlabel=r'$\mathrm{iteration}$ $k$', 
                            ylabel='x distance',
                            yscale='log')

np.set_printoptions(formatter={'float':lambda x: f' {x:.2e}' if x>0 else f'{x:.2e}'})
for alpha in alpha_choice:
    for rho in rho_choice:
        
        # ci = alpha_choice.index(alpha)
        # lsi = rho_choice.index(rho)
        ci = rho_choice.index(rho)
        lsi = alpha_choice.index(alpha)
        c = color[ci]
        ls = linestyle[lsi]
        
        # if True:
        if rho != 5:
            alg = IPLUX(prob, network, alpha=alpha, rho=rho, verbose=False)
            for i in range(MAX_ITER):
                alg.step()
        
        prefix = f'{alg.name}_a{alpha}_r{rho}'
        filename_oe = f'log/N{num_node}/{prefix}_oe.txt'
        filename_cv = f'log/N{num_node}/{prefix}_cv.txt'
        filename_xd = f'log/N{num_node}/{prefix}_xd.txt'
        
        obj_err_figure.add_line_file(prefix, filename_oe, style=c+ls)
        cons_vio_figure.add_line_file(prefix, filename_cv, style=c+ls)
        x_dis_figure.add_line_file(prefix, filename_xd, style=c+ls)
            
        obj_err_figure.paint(MAX_ITER=MAX_ITER)
        cons_vio_figure.paint(MAX_ITER=MAX_ITER, nonnegy=True)
        x_dis_figure.paint(MAX_ITER=MAX_ITER, nonnegy=True)

self.prob minimize param11378 @ var11377 + norm1(var11377) + 2.5 @ QuadForm(var11377 + -param11379, [[1.00 0.00 0.00]
 [0.00 1.00 0.00]
 [0.00 0.00 1.00]]) + 0.5 @ quad_over_lin(param11380 @ var11377, 1.0) + param11381 @ var11377 + param11382[0] @ QuadForm(var11377, [[1.00 0.00 0.00]
 [0.00 1.00 0.00]
 [0.00 0.00 1.00]]) + Promote(-2.0, (3,)) @ param11383 @ var11377
subject to QuadForm(var11377 + -param11385, [[1.00 0.00 0.00]
 [0.00 1.00 0.00]
 [0.00 0.00 1.00]]) <= param11386[0]
reset
IPLUX alpha 5.0 rho 1, iter 0, obj err: 1.79e+00, cons vio: 0.00e+00
time 0.02, saved

IPLUX alpha 5.0 rho 1, iter 100, obj err: 2.45e-02, cons vio: 7.16e-02
time 5.18, saved

IPLUX alpha 5.0 rho 1, iter 200, obj err: 5.63e-03, cons vio: 2.03e-02
time 10.14, saved

IPLUX alpha 5.0 rho 1, iter 300, obj err: 2.02e-03, cons vio: 5.34e-03
time 14.98, saved

IPLUX alpha 5.0 rho 1, iter 400, obj err: 2.56e-04, cons vio: 1.32e-03
time 19.75, saved

IPLUX alpha 5.0 rho 1, iter 500, obj err: 7.70e-05, cons vio: 

In [5]:
# UDC

# best parameters:
# UDC_pt: a20_r4, 156.29s
# UDC_PEXTRA: a0_r1
# UDC_DistADMM: a0_r2


MAX_ITER = 2000

log_dir = 'log'
if not os.path.exists(log_dir):
    os.makedirs(log_dir)

# rho_choice = [0.1, 1, 10]
# rho_choice = [0.5, 1, 2, 4]
rho_choice = [1]

# alpha_choice = [0, 10, 100]
# alpha_choice = [0, 0.1]
alpha_choice = [0]

color = ['b', 'g', 'r', 'c', 'm', 'y', 'k']
linestyle = ['-', '--', '-.', ':', '']

obj_err_figure = MyFigure(filename='obj_err', 
                            xlabel=r'$\mathrm{iteration}$ $k$', 
                            ylabel='objective error',
                            yscale='log')
cons_vio_figure = MyFigure(filename='cons_vio', 
                            xlabel=r'$\mathrm{iteration}$ $k$', 
                            ylabel='constraint violation',
                            yscale='log')
x_dis_figure = MyFigure(filename='x_distance', 
                            xlabel=r'$\mathrm{iteration}$ $k$', 
                            ylabel='x distance',
                            yscale='log')
    
np.set_printoptions(formatter={'float':lambda x: f' {x:.2e}' if x>0 else f'{x:.2e}'})
for alpha in alpha_choice:
    for rho in rho_choice:
        # ci = alpha_choice.index(alpha)
        # lsi = rho_choice.index(rho)
        ci = rho_choice.index(rho)
        lsi = alpha_choice.index(alpha)
        c = color[ci]
        ls = linestyle[lsi]
        
        if True:
        # if alpha != 0:
        # if rho != 1:
        # if rho == 4:
            # alg = UDC(prob, network, rho=rho, alpha=alpha,
            #         param_setting='pt', verbose=False)
            alg = UDC(prob, network, rho=rho, alpha=alpha,
                      param_setting='PEXTRA', verbose=False)
            # alg = UDC(prob, network, rho=rho, alpha=alpha,
            #           param_setting='DistADMM', verbose=False)
            for i in range(MAX_ITER):
                alg.step()
            
        prefix = f'{alg.name}_a{alpha}_r{rho}'
        filename_oe = f'log/N{num_node}/{prefix}_oe.txt'
        filename_cv = f'log/N{num_node}/{prefix}_cv.txt'
        filename_xd = f'log/N{num_node}/{prefix}_xd.txt'
        
        obj_err_figure.add_line_file(prefix, filename_oe, style=c+ls)
        cons_vio_figure.add_line_file(prefix, filename_cv, style=c+ls)
        x_dis_figure.add_line_file(prefix, filename_xd, style=c+ls)
            
        obj_err_figure.paint(MAX_ITER=MAX_ITER)
        cons_vio_figure.paint(MAX_ITER=MAX_ITER, nonnegy=True)
        x_dis_figure.paint(MAX_ITER=MAX_ITER, nonnegy=True)


UDC setting: PEXTRA
self.prob minimize quad_over_lin(param2271 @ var2270, 1.0) + param2272 @ var2270 + param2273[0] @ norm1(var2270) + 0.0 @ quad_over_lin(Promote(param2274[0], (3,)) @ var2270 + -param2275, 1.0) + 0.5 @ quad_over_lin(maximum(param2276 + quad_over_lin(var2270 + -param2278, 1.0) + -param2279, 0.0), 1.0) + 0.5 @ quad_over_lin(param2277 + param2282 @ var2270, 1.0)
subject to QuadForm(var2270 + -param2280, [[1.00 0.00 0.00]
 [0.00 1.00 0.00]
 [0.00 0.00 1.00]]) <= param2281[0]
reset
UDC_PEXTRA alpha 0 rho 1, iter 0, obj err: 1.79e+00, cons vio: 0.00e+00
time 0.01, saved

UDC_PEXTRA alpha 0 rho 1, iter 100, obj err: 2.04e-02, cons vio: 6.74e-02
time 5.76, saved

UDC_PEXTRA alpha 0 rho 1, iter 200, obj err: 3.74e-03, cons vio: 1.32e-02
time 11.31, saved

UDC_PEXTRA alpha 0 rho 1, iter 300, obj err: 7.01e-04, cons vio: 1.97e-03
time 17.54, saved

UDC_PEXTRA alpha 0 rho 1, iter 400, obj err: 8.88e-05, cons vio: 2.21e-04
time 23.04, saved

UDC_PEXTRA alpha 0 rho 1, iter 500, obj

In [34]:
H2 = 0.5 * (np.identity(num_node)-network)
np.linalg.eigvals(H2)
# scipy.linalg.sqrtm(H2)

array([ 6.20e-01,  5.67e-01,  5.37e-01,  4.89e-01,  4.45e-01,  3.76e-01,
        3.64e-01,  3.08e-01,  2.75e-01,  1.87e-01,  1.22e-01, -1.47e-18,
        1.33e-02,  3.16e-02,  4.72e-02,  5.54e-02,  4.58e-01,  7.23e-02,
        8.33e-02,  7.14e-02])

In [58]:
# DPMM
# best parameters: a0.1_r1_t1.5

from plot_utils import MyFigure

alg_name = 'DPMM'
MAX_ITER = 2000

# theta_choice = [0.5, 1, 1.5, 1.9]
# theta_choice = [0.1, 1.9]
theta_choice = [1.5]

# rho_choice = [0.1, 1, 10]
# rho_choice = [0.5, 1, 2]
rho_choice = [1]

# alpha_choice = [0, 10, 100]
# alpha_choice = [0.1, 1, 10]
alpha_choice = [0.1]

color = ['b', 'g', 'r', 'c', 'm', 'y', 'k']
linestyle = ['-', '--', '-.', ':', '']

obj_err_figure = MyFigure(filename='obj_err', 
                            xlabel=r'$\mathrm{iteration}$ $k$', 
                            ylabel='objective error',
                            yscale='log')
cons_vio_figure = MyFigure(filename='cons_vio', 
                            xlabel=r'$\mathrm{iteration}$ $k$', 
                            ylabel='constraint violation',
                            yscale='log')
x_dis_figure = MyFigure(filename='x_distance', 
                            xlabel=r'$\mathrm{iteration}$ $k$', 
                            ylabel='x distance',
                            yscale='log')


for alpha in alpha_choice:
    for rho in rho_choice:
        ci = alpha_choice.index(alpha)
        lsi = rho_choice.index(rho)
        # ci = rho_choice.index(rho)
        # lsi = alpha_choice.index(alpha)
        c = color[ci]
        ls = linestyle[lsi]
        
        for theta in theta_choice:
            
            if True:
            # if rho != 1:
            # if alpha != 0:
            # if theta == 1.9:
                alg = UDC(prob, network, rho=rho, alpha=alpha, theta=theta,
                            param_setting='DPMM', verbose=False)
                for i in range(MAX_ITER):
                    alg.step()

            # plot
            # ci = theta_choice.index(theta)
            # c = color[ci]
            
            prefix = f'{alg_name}_a{alpha}_r{rho}_t{theta}'
            filename_oe = f'log/N{num_node}/{prefix}_oe.txt'
            filename_cv = f'log/N{num_node}/{prefix}_cv.txt'
            filename_xd = f'log/N{num_node}/{prefix}_xd.txt'
            
            obj_err_figure.add_line_file(prefix, filename_oe, style=c+ls)
            cons_vio_figure.add_line_file(prefix, filename_cv, style=c+ls)
            x_dis_figure.add_line_file(prefix, filename_xd, style=c+ls)
                
            obj_err_figure.paint(MAX_ITER=MAX_ITER)
            cons_vio_figure.paint(MAX_ITER=MAX_ITER, nonnegy=True)
            x_dis_figure.paint(MAX_ITER=MAX_ITER, nonnegy=True)


UDC setting: DPMM
UDC setting: PEXTRA
self.prob minimize quad_over_lin(param30818 @ var30817, 1.0) + param30819 @ var30817 + param30820[0] @ norm1(var30817) + 0.05 @ quad_over_lin(Promote(param30821[0], (3,)) @ var30817 + -param30822, 1.0) + 0.5 @ quad_over_lin(maximum(param30823 + quad_over_lin(var30817 + -param30825, 1.0) + -param30826, 0.0), 1.0) + 0.5 @ quad_over_lin(param30824 + param30829 @ var30817, 1.0)
subject to QuadForm(var30817 + -param30827, [[1.00 0.00 0.00]
 [0.00 1.00 0.00]
 [0.00 0.00 1.00]]) <= param30828[0]
reset
DPMM alpha 0.1 rho 1 theta 1.5, iter 0, obj err: 4.57e-01, cons vio: 0.00e+00
time 0.02, saved



c:\Users\xtq\OneDrive - shanghaitech.edu.cn\Research\Dual AMM\numerical\exp3_synthetic_IPLUX\algorithms\Liu25_UDC.py:437: ComplexWarning: Casting complex values to real discards the imaginary part
  AyHz_mu_k[i] = self.y_mu_cur.T @ self.A_weight[i] \
c:\Users\xtq\OneDrive - shanghaitech.edu.cn\Research\Dual AMM\numerical\exp3_synthetic_IPLUX\algorithms\Liu25_UDC.py:439: ComplexWarning: Casting complex values to real discards the imaginary part
  AyHz_lam_k[i] = self.y_lam_cur.T @ self.A_weight[i] \
c:\Users\xtq\OneDrive - shanghaitech.edu.cn\Research\Dual AMM\numerical\exp3_synthetic_IPLUX\algorithms\Liu25_UDC.py:465: ComplexWarning: Casting complex values to real discards the imaginary part
  Hy_mu_nxt[i] = self.y_mu_nxt.T @ self.H2_half[i]
c:\Users\xtq\OneDrive - shanghaitech.edu.cn\Research\Dual AMM\numerical\exp3_synthetic_IPLUX\algorithms\Liu25_UDC.py:466: ComplexWarning: Casting complex values to real discards the imaginary part
  Hy_lam_nxt[i] = self.y_lam_nxt.T @ self.H2_half[i

DPMM alpha 0.1 rho 1 theta 1.5, iter 100, obj err: 1.39e-02, cons vio: 3.29e-02
time 8.13, saved

DPMM alpha 0.1 rho 1 theta 1.5, iter 200, obj err: 5.09e-04, cons vio: 1.16e-03
time 16.26, saved

DPMM alpha 0.1 rho 1 theta 1.5, iter 300, obj err: 5.93e-04, cons vio: 8.95e-04
time 24.25, saved

DPMM alpha 0.1 rho 1 theta 1.5, iter 400, obj err: 3.26e-04, cons vio: 3.32e-04
time 31.76, saved

DPMM alpha 0.1 rho 1 theta 1.5, iter 500, obj err: 7.09e-05, cons vio: 8.17e-05
time 39.17, saved

DPMM alpha 0.1 rho 1 theta 1.5, iter 600, obj err: 1.64e-05, cons vio: 1.49e-05
time 46.64, saved

DPMM alpha 0.1 rho 1 theta 1.5, iter 700, obj err: 2.04e-05, cons vio: 1.73e-05
time 54.96, saved

DPMM alpha 0.1 rho 1 theta 1.5, iter 800, obj err: 7.84e-06, cons vio: 8.49e-06
time 63.27, saved

DPMM alpha 0.1 rho 1 theta 1.5, iter 900, obj err: 4.12e-07, cons vio: 1.73e-06
time 70.99, saved

DPMM alpha 0.1 rho 1 theta 1.5, iter 1000, obj err: 1.39e-06, cons vio: 1.11e-06
time 79.23, saved

DPMM alpha

In [ ]:
# dual subgradient
# best parameters: g10

from plot_utils import MyFigure

alg_name = 'dual_subgradient'
MAX_ITER = 2000
# gamma_choice = [0.1, 1, 5, 25]
# gamma_choice = [2.5, 5, 10, 50]
gamma_choice = [5, 10]
color = ['b', 'g', 'r', 'c', 'm', 'y', 'k']

obj_err_figure = MyFigure(filename='obj_err', 
                            xlabel=r'$\mathrm{iteration}$ $k$', 
                            ylabel='objective error',
                            yscale='log')
cons_vio_figure = MyFigure(filename='cons_vio', 
                            xlabel=r'$\mathrm{iteration}$ $k$', 
                            ylabel='constraint violation',
                            yscale='log')
x_dis_figure = MyFigure(filename='x_distance', 
                            xlabel=r'$\mathrm{iteration}$ $k$', 
                            ylabel='x distance',
                            yscale='log')


for gamma in gamma_choice:
    if True:
    # if gamma == 50:
        alg = dual_subgradient(prob, network, gamma)
        for i in range(MAX_ITER):
            alg.step()

    # plot
    ci = gamma_choice.index(gamma)
    c = color[ci]
    
    prefix = f'{alg_name}_g{gamma}'
    filename_oe = f'log/N{num_node}/{prefix}_oe.txt'
    filename_cv = f'log/N{num_node}/{prefix}_cv.txt'
    filename_xd = f'log/N{num_node}/{prefix}_xd.txt'
    
    obj_err_figure.add_line_file(prefix, filename_oe, style=c)
    cons_vio_figure.add_line_file(prefix, filename_cv, style=c)
    x_dis_figure.add_line_file(prefix, filename_xd, style=c)
        
    obj_err_figure.paint(MAX_ITER=MAX_ITER)
    cons_vio_figure.paint(MAX_ITER=MAX_ITER, nonnegy=True)
    x_dis_figure.paint(MAX_ITER=MAX_ITER, nonnegy=True)

    

self.prob minimize quad_over_lin(param32217 @ var32216, 1.0) + param32218 @ var32216 + norm1(var32216) + param32219[0] @ quad_over_lin(var32216, 1.0) + Promote(-2.0, (3,)) @ param32220 @ var32216 + param32221 @ var32216
subject to QuadForm(var32216 + -param32224, [[1.00 0.00 0.00]
 [0.00 1.00 0.00]
 [0.00 0.00 1.00]]) <= param32225[0]
reset
dual subgradient gamma 5, iter 0, obj err: 4.57e-01, cons vio: 0.00e+00
time 0.01, saved

dual subgradient gamma 5, iter 100, obj err: 4.50e-01, cons vio: 5.38e-01
time 6.47, saved

dual subgradient gamma 5, iter 200, obj err: 3.67e-01, cons vio: 3.97e-01
time 12.63, saved

dual subgradient gamma 5, iter 300, obj err: 3.23e-01, cons vio: 3.37e-01
time 19.47, saved

dual subgradient gamma 5, iter 400, obj err: 2.94e-01, cons vio: 3.02e-01
time 25.85, saved

dual subgradient gamma 5, iter 500, obj err: 2.60e-01, cons vio: 2.68e-01
time 32.24, saved

dual subgradient gamma 5, iter 600, obj err: 2.41e-01, cons vio: 2.48e-01
time 38.70, saved

dual subgr

array([0., 1., 0., 0.])

In [28]:
# compare fine-tuned algorithms

# instance 1
param_best_list = [['IPLUX', 5.0, 5], 
                 ['UDC_pt', 0, 2], 
                 ['UDC_PEXTRA', 0, 4], 
                 ['UDC_DistADMM', 0, 1],
                 ['DPMM', 0, 4, 1.5],
                 ['dual_subgradient', 5]]
alg_list = []
alg_list.append(IPLUX(prob, network, alpha=5.0, rho=5))
alg_list.append(UDC(prob, network, alpha=0, rho=2, param_setting='pt'))
alg_list.append(UDC(prob, network, alpha=0, rho=4, param_setting='PEXTRA'))
alg_list.append(UDC(prob, network, alpha=0, rho=1, param_setting='DistADMM'))
alg_list.append(UDC(prob, network, alpha=0, rho=4, theta=1.5, param_setting='DPMM'))
alg_list.append(dual_subgradient(prob, network, gamma=5))


# instance 2
# param_best_list = [['IPLUX', 4.0, 1], 
#                  ['UDC_pt', 20, 4], 
#                  ['UDC_PEXTRA', 0.1, 1], 
#                  ['UDC_DistADMM', 0.1, 2],
#                  ['DPMM', 0.1, 1, 1.5],
#                  ['dual_subgradient', 10]]
# alg_list = []
# alg_list.append(IPLUX(prob, network, alpha=4.0, rho=1))
# alg_list.append(UDC(prob, network, alpha=20, rho=4, param_setting='pt'))
# alg_list.append(UDC(prob, network, alpha=0.1, rho=1, param_setting='PEXTRA'))
# alg_list.append(UDC(prob, network, alpha=0.1, rho=2, param_setting='DistADMM'))
# alg_list.append(UDC(prob, network, alpha=0.1, rho=1, theta=1.5, param_setting='DPMM'))
# alg_list.append(dual_subgradient(prob, network, gamma=10))




color = ['b', 'g', 'r', 'c', 'm', 'y', 'k']
MAX_ITER = 2000

obj_err_figure = MyFigure(filename='compare_obj_err', 
                            xlabel=r'$\mathrm{Iteration}$ $k$', 
                            ylabel=r'$\mathrm{Objective}\ \mathrm{Error}$')

cons_vio_figure = MyFigure(filename='compare_cons_vio', 
                            xlabel=r'$\mathrm{Iteration}$ $k$', 
                            ylabel=r'$\mathrm{Constraint}\ \mathrm{Violation}$')

x_dis_figure = MyFigure(filename='compare_x_distance', 
                            xlabel=r'$\mathrm{Iteration}$ $k$', 
                            ylabel=r'$\Vert \mathbf{x}^k - \mathbf{x}^\star \Vert$')

for alg in alg_list:
    ci = alg_list.index(alg)
    c = color[ci]
    
    # for i in range(MAX_ITER):
    #     alg.step()
    # log_dir = 'log/N20'
    # filename_oe = f'{log_dir}/{alg.file_prefix}_oe.txt'
    # filename_cv = f'{log_dir}/{alg.file_prefix}_cv.txt'
    # filename_xd = f'{log_dir}/{alg.file_prefix}_xd.txt'
    
    # instance 1
    log_dir = f'instance/instance1_N20E40/log'
    file_alg_name = alg.name.replace(' ', '_').replace('-', '_')
    filename_oe = f'{log_dir}/{file_alg_name}/{alg.file_prefix}_oe.txt'
    filename_cv = f'{log_dir}/{file_alg_name}/{alg.file_prefix}_cv.txt'
    filename_xd = f'{log_dir}/{file_alg_name}/{alg.file_prefix}_xd.txt'
    
    # instance 2
    # log_dir = f'instance/instance2_N20E26/log'
    # file_alg_name = alg.name.replace(' ', '_').replace('-', '_')
    # filename_oe = f'{log_dir}/{file_alg_name}/all_log/{alg.file_prefix}_oe.txt'
    # filename_cv = f'{log_dir}/{file_alg_name}/all_log/{alg.file_prefix}_cv.txt'
    # filename_xd = f'{log_dir}/{file_alg_name}/all_log/{alg.file_prefix}_xd.txt'
    
    obj_err_figure.add_line_file(alg.name, filename_oe, style=c)
    cons_vio_figure.add_line_file(alg.name, filename_cv, style=c)
    x_dis_figure.add_line_file(alg.name, filename_xd, style=c)

    obj_err_figure.paint(MAX_ITER=MAX_ITER)
    cons_vio_figure.paint(MAX_ITER=MAX_ITER, nonnegy=True)
    x_dis_figure.paint(MAX_ITER=MAX_ITER, nonnegy=True)

self.prob minimize param8117 @ var8116 + norm1(var8116) + 2.5 @ QuadForm(var8116 + -param8118, [[1.00 0.00 0.00]
 [0.00 1.00 0.00]
 [0.00 0.00 1.00]]) + 0.1 @ quad_over_lin(param8119 @ var8116, 1.0) + param8120 @ var8116 + param8121[0] @ QuadForm(var8116, [[1.00 0.00 0.00]
 [0.00 1.00 0.00]
 [0.00 0.00 1.00]]) + Promote(-2.0, (3,)) @ param8122 @ var8116
subject to QuadForm(var8116 + -param8124, [[1.00 0.00 0.00]
 [0.00 1.00 0.00]
 [0.00 0.00 1.00]]) <= param8125[0]
reset
IPLUX alpha 5.0 rho 5, iter 0, obj err: 1.79e+00, cons vio: 0.00e+00
time 0.02, saved

UDC setting: pt
self.prob minimize quad_over_lin(param8157 @ var8156, 1.0) + param8158 @ var8156 + param8159[0] @ norm1(var8156) + 0.0 @ quad_over_lin(Promote(param8160[0], (3,)) @ var8156 + -param8161, 1.0) + 0.5 @ quad_over_lin(maximum(param8162 + quad_over_lin(var8156 + -param8164, 1.0) + -param8165, 0.0), 1.0) + 0.5 @ quad_over_lin(param8163 + param8168 @ var8156, 1.0)
subject to QuadForm(var8156 + -param8166, [[1.00 0.00 0.00]
 

In [71]:

# ============================= trace in x1 x1 plane =========================
import matplotlib.pyplot as plt
import matplotlib.cm as cm

fig, ax = plt.subplots()     # Create a figure containing a single Axes.

delta = 0.05
x = np.arange(0.0, 1, delta)
y = np.arange(0.0, 1, delta)
X, Y = np.meshgrid(x, y)
F = X*alg.c[0] + Y*alg.c[1]
G = -alg.d[0]*np.log(1+X) - alg.d[1]*np.log(1+Y) + alg.b[0]*2/num_node

im = ax.imshow(F, interpolation='bilinear', origin='lower',
            cmap=cm.gray, extent=(0, 1, 0, 1))
CS_F = ax.contour(X, Y, F, 5)
ax.clabel(CS_F, fontsize=50)

# if there are only two nodes, we can polt the constraint
if num_node == 2:
    CS_G = ax.contour(X, Y, G, colors='k')
    ax.clabel(CS_G, fontsize=20)

x_log = np.array(alg.x_log)
x_avg_log = np.array(alg.x_avg_log)
ax.scatter(alg.x_star[0], alg.x_star[1], s=5000, c='r', marker='X', label='optimal')
ax.scatter(x_log[-1,0], x_log[-1,1], s=500, c='y', marker='^', label='last')
ax.scatter(x_log[:,0], x_log[:,1], label='iter', s=100, c='b', marker='^')  # Plot some data on the Axes.
ax.scatter(x_avg_log[:,0], x_avg_log[:,1], label='iter avg', s=100, marker='^')

ax.set_title('x position')
ax.set_xlabel('$x_0$')
ax.set_ylabel('$x_1$')
ax.legend()
fig.savefig('x.png')
plt.close()


'1'